**Como primer punto se enlaza al drive para obtener el dataset y asi poder realizar un cuadernillo.
El dataset que se incluye en este cuadernillo es el de "Secuencias de proteínas estructurales",se quiere obtener una comprensión clara y concisa de la función y relevancia del PDB en la investigación y educación en biología estructural. También se busca resaltar la importancia de los métodos utilizados para determinar las estructuras de proteínas y otras macromoléculas biológicas, así como la diversidad de información disponible en el PDB.





In [ ]:
import numpy as np
import pandas as pd
import warnings
from google.colab import drive
from matplotlib import pyplot
from scipy import optimize
%matplotlib inline

En esta primer cuadro de codigo se importan los datos que tiene el dataset.


In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AquÍ fue lo que hizo fue importacion de los dataset en el drive

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Dataset Machine learning/train.csv")
data = data.fillna(0)
data = np.array(data)
x = data[:36800,:47]
y = data[:36800,49]
m_ejemplos = y.size
print(x)
print(y)

[[0.179546 0.054544 0.041338 ... -3.768853 -7.064849 -44.878769]
 [0.109351 0.007854 0.006311 ... 0.001645 0.004429 0.001218]
 [0.100744 0.007863 0.0055 ... 0.008327 0.002209 0.016153]
 ...
 [0.116995 0.018328 0.032508 ... 0.007924 0.000872 -0.004522]
 [0.11473 0.013457 0.023524 ... 0.011838 0.000443 -0.010801]
 [0.110254 0.00404 0.007224 ... -0.000864 -0.000704 -0.005676]]
[0 0 0 ... 1 1 1]


Se importo la ruta de los dataset.
Donde el array viene a ser lista de datos donde se va a convertir un vector.
donde las variables de  "x" viene a ser los numeros de caracteristicas en un rango donde el numero 30 es el limite y la variable "y" vendria a ser lo ejemplos del dataset.

Donde tambien se trabajo el 80% de los datos para entrenas y el 20% para aprobar la efectividad, como se ve mas adelante.

In [ ]:
x.shape

(36800, 47)

x.shape me permite ver el indice de columnas y filas.

In [ ]:
for i in range (x.shape[0]):
  if (y[i] == 0):
    y[i] = 3;

In [ ]:
print(y)

[3 3 3 ... 1 1 1]


In [ ]:
input_layer_size  = 47
num_labels = 3


con la funcion sigmoidea, se procesa los datos para obtener un resultado entre 0 y 1

In [ ]:
def normalizarX(x):
  x_normalizada = []
  promedio = np.mean(x,axis=0)
  x = x.astype(float)
  desviacion_estandar = np.std(x,axis=0)
  for i in range(m_ejemplos):
    caracteristicas_x = []
    for j in range(x.shape[1]):
      if desviacion_estandar[j] != 0:
        caracteristicas_x.append((x[i,j]-promedio[j])/desviacion_estandar[j])
      else:
        caracteristicas_x.append(0)
    x_normalizada.append(caracteristicas_x)
  return x_normalizada, promedio, desviacion_estandar

Al normalizar hace que los valores sean mas pequeños.
donde tambien saca el promedio de las colunmas de "x" para asi no tener valores de 0 o error.

In [ ]:
x_normalizada, promedio, desviacion_estandar = normalizarX(x)
x_normalizada = np.array(x_normalizada)
x = np.concatenate((np.ones((m_ejemplos,1)),x_normalizada),axis=1)

A quí se hace las menciones de las caracteristicas para asi poderse guardar.

In [ ]:
def sigmoid(z):
  z = np.array(z)
  g = np.zeros(z.shape)
  g = 1/(1+np.exp(-z))
  return g

In [ ]:
def lrCostFunction(theta, x, y, lambda_):
    if y.dtype == bool:
        y = y.astype(int)
    J = 0
    grad = np.zeros(theta.shape)
    h = sigmoid(x.dot(theta.T))
    temp = theta
    temp[0] = 0
    J = (1 / m_ejemplos) * (np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))) + (lambda_ / (2 * m_ejemplos)) * np.sum(np.square(temp))
    grad = (1 / m_ejemplos) * (h - y).dot(x)
    grad = grad + (lambda_ / m_ejemplos) * temp
    return J, grad

In [ ]:
def oneVsAll(x, y, num_labels, lambda_):
    m, n = x.shape
    all_theta = np.zeros((num_labels, n + 1))
    x = np.concatenate([np.ones((m, 1)), x], axis=1)
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (x, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)
        all_theta[c] = res.x
    return all_theta

In [ ]:
lambda_ = 0.0001
warnings.filterwarnings("ignore")
all_theta = oneVsAll(x, y, num_labels, lambda_)

Con el costo se puede ver cada interaccion que se pueda ajustar a taza de proximidad

In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    p = np.zeros(m)
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
    return p

In [ ]:
print(x.shape)
pred = predictOneVsAll(all_theta, x)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = x[10:150, :].copy()
print(XPrueba.shape)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)
print(y[10:150])

(36800, 48)
Precision del conjuto de entrenamiento: 32.02%
(140, 48)
(140, 49)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [1]:
def entrenar_regresion_logistica_multivariable_con_reg(X_train, y_train, tasa_aprendizaje, lambda_param, num_iteraciones):
    m, n = X_train.shape
    num_clases = len(np.unique(y_train))
    X_train_con_intercepto = agregar_intercepto(X_train)  # Agregar intercepto a X_train

    thetas = np.zeros((num_clases, n + 1))  # Inicializar parámetros

    for clase in range(1, num_clases + 1):
        y_binario = (y_train == clase).astype(int)

        theta = thetas[clase - 1]

        for i in range(num_iteraciones):
            z = np.dot(X_train_con_intercepto, theta)
            h = sigmoid(z)
            error = h - y_binario
            gradiente = np.dot(X_train_con_intercepto.T, error) / m
            gradiente[1:] += (lambda_param / m) * theta[1:]  # Regularización, excepto para el término de intercepto
            theta -= tasa_aprendizaje * gradiente

        thetas[clase - 1] = theta

    return thetas

def agregar_intercepto(X):
    m = X.shape[0]
    intercepto = np.ones((m, 1))
    return np.concatenate((intercepto, X), axis=1)